
# Tutorial: babI6 Training and Preprocessing in Python

In this tutorial, we will try to implement question asking and answering for babI6 tasks. babI6 tasks are tasks where we supply a text that describes movement of people in different locations and ask questions about the locations of said people while they are moving around. More on the babI tasks can be in this [paper](https://arxiv.org/abs/1502.05698) and this [repository](https://github.com/facebookarchive/bAbI-tasks?tab=readme-ov-file) by Facebook.

In [1]:
from pathlib import Path
from typing import Tuple, List
from lambeq.experimental.discocirc import DisCoCircReader
import os
import warnings
import pickle

warnings.filterwarnings('ignore')
os.environ['TOKENIZERS_PARALLELISM'] = 'true'

/Users/ouissal.moumou/actual_discocirc/notebooks/discocirc-experiments/lambeq/experimentsenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Before we delve into the code, we first highlight two new features of the new parser that will be used in this tutorial: the sandwich functor and foliated frames. 

In the previous versions of the parser, the semantic functor, while providing quantum implementations for boxes, wires and states, did not specify the quantum implementation of frames. The sandwich functor addresses this issue by introducing a novel construction that breaks down a frame into a sequence of boxes with the frame's contents. Now that we have these different frames, we can decide whether we want every layer in these frames to be assigned their operator or have the same operator for all the layers (different parameters assigned to the layers as opposed to having all the layers having the same parameter). For more detail on this, we recommend reading the paper explaining  the theory behind the new parser [here](in_the_making).

## 1. Setting Up Configuration Variables

This cell defines paths and key configuration variables:
- `FILEPATH` specifies paths to the file containing the babI6 data. 
- `TEXT_LENGTH` specifies the maximum number of sentences in a context for the experiment.
- `MAX_WIDTH` specifies the maximum number of wires in a circuit for the experiment.
- `SANDWICH` is a flag for using the sandwich functor: True to apply the sandwich functor on the circuits, False to apply the usual semantic functor.
- `FFL` is a flag for activating the foliated frames. True to set different parameters for the different layers of frames, False to set the same parameter for all the layers. It is to note that it only makes sense to have this flag if the sandwich functor is activated.
- `TRAINING_SAMPLE_SIZE`, `VALIDATION_SAMPLE_SIZE`, and `TEST_SAMPLE_SIZE` specify the size of the data for training, validation, and testing.

In [ ]:
# Here we store all the variables needed for the rest of the code: file paths, configurations, parameters...

# The path of the file where the initial babI6 data is stored
FILEPATH = '../data/qa6_train_10k.txt'

# Maximum length of the context
TEXT_LENGTH = 4

# Maximum Number of wires in a circuit
MAX_WIDTH = 9

# SANDWICH functor flag
SANDWICH = True

# Updating the FFL parameter
FFL = True

# Sizes of training, validation, and test datasets
TRAINING_SAMPLE_SIZE = 120
VALIDATION_SAMPLE_SIZE = 30
TEST_SAMPLE_SIZE = 30

# Paths of resulting files for the datasets
TRAINING_DATASET_FILEPATH = 'tutorial_training_ffl' + str(FFL) + '_sandwich_' + str(SANDWICH) + '.pkl'
VALIDATION_DATASET_FILEPATH = 'tutorial_validation_ffl' + str(FFL) + '_sandwich_' + str(SANDWICH) + '.pkl'
TEST_DATASET_FILEPATH = 'tutorial_test_ffl' + str(FFL) + '_sandwich_' + str(SANDWICH) + '.pkl'

## 2. Data Preprocessing Function

The next step is to write a function `task_file_reader`, which processes the babI6 dataset and returns a list of texts, a list of questions on these texts, a list of answers to these questions, and a list of the lengths of the texts. This function reads and cleans lines from the `FILEPATH`, splits lines into stories, and extracts text sentences, questions, and answers.

After extracting the texts, they are filtered to only keep the ones whose number of sentences is less than or equal to `TEXT_LENGTH`, which we set in the previous cell to determine the maximum number of sentences that we want in a text for optimization purposes. We want to make sure that we do not get huge circuits later on when we convert the texts into circuits, which might slow down the experiment. 

After this filtering, the last step is to convert the list of texts from a list of arrays of sentences, to a list of sentences. In other words, we concatenate the sentences in each text (which is an array) to obtain a string.

In [3]:
# Reading the texts, questions, expected answers, and text_length from the TXT file
def task_file_reader(path : str | Path) -> Tuple[List[List[str]],
                                                       List[str],
                                                       List[str],
                                                       List[str]]:
    """
    reads the .txt file at `path`
    returns 3 lists of equal length
    - text sentences, questions, answers and text length
    """
    with open(path) as f:
        lines = f.readlines()

    # split the lines into stories
    # record the first line location of new stories
    story_splits = [i for i, line in enumerate(lines) if line[0:2] == '1 ']
    # have no more need for line indices - delete these
    lines = [' '.join(line.split(' ')[1:]) for line in lines]
    # also delete . and \n
    lines = [line.replace('.', '').replace('\n','') for line in lines]
    stories = [lines[i:j] for i, j in zip(story_splits, story_splits[1:]+[None])]

    # create text and QnA pairs
    texts = []
    qnas = []
    text_length = []
    for story in stories:
        # record the lines in the story corresponding to questions
        question_splits = [i for i, line in enumerate(story) if '?' in line]
        for index in question_splits:
            # record the text corresponding to each question
            ctx = [line.lower() for line in story[:index] if '?' not in line]
            texts.append(ctx)
            text_length.append(len(ctx))
            # record the question
            qnas.append(story[index])

    # split qna into questions and answers
    questions = [qna.split('\t')[0].lower().rstrip()[:-1] + " ?" for qna in qnas]
    answers = [qna.split('\t')[1].lower() for qna in qnas]

    # Filtering the data   
    filtered_data = [
    (text, question, answer, text_length)
    for text, question, answer, text_length in zip(texts, questions, answers, text_length)
    if len(text) <= TEXT_LENGTH
    ]

    # Applying the filter
    texts, questions, answers, text_length = map(list, zip(*filtered_data))

    # Converting the texts from arrays of sentences to strings
    processed_texts_list = []
    for text in texts:
        processed_text = ""
        for sentence in text:
            processed_text += sentence + ". "
                
        processed_texts_list.append(processed_text)
    

    return processed_texts_list, questions, answers, text_length


texts, questions, answers, text_lengths = task_file_reader(FILEPATH)

## 3. Converting The Texts into Circuits

Now that we have our texts and the rest of data ready and pre-processed, we move on to the crucial step of converting them into circuits. We first start by initializing the reader, then we use it with the `SANDWICH` flag indicating whether to use the SANDWICH functor or not, as well as the `FFL` flag which indicates whether to assign different parameters to the different layers of the frame, or the same parameters.

Moreover, we store the data in a dictionary where each entry includes the text, the corresponding generated DisCoCirc circuit, the question, the answer, and the text_length.

In [ ]:
# making the circuits from the texts and storing them in the dictionary
reader = DisCoCircReader()
datadict = {}
for i, (text, quest, ans, text_length) in enumerate(zip(texts, questions, answers, text_lengths)):
    datadict.update({i:{'text':text, 'dsc_diag': reader.text2circuit(text, sandwich=SANDWICH, foliated_frame_labels = FFL),  'question':quest, 'answer':ans, 'text_length': text_length}})
    

## 4. Converting The Circuits from DisCoCirc Circuits to Quantum Circuits
While we have the circuits corresponding to the texts ready, they are still DisCoCirc circuits, not quantum circuits. Therefore, we need to convert the DisCocirc circuits into quantum circuits by applying an ansatz. In this case, we chose to apply the Sim4Ansatz with 3 layers, and one qubit for each noun.

In [ ]:
from lambeq import Sim4Ansatz
from lambeq import AtomicType

N = AtomicType.NOUN
ansatz = Sim4Ansatz({N:1}, n_layers=3)

for i in datadict.keys():
    datadict[i].update({'text_circuit_sim4_13': ansatz(datadict[i]['dsc_diag'])})

KeyboardInterrupt: 

## 5. Question Asking Circuits and Further Processing of the Circuits
The main spirit of this tutorial is having questions also be circuits that one sequetially composes with the circuits representing texts in order to see the similarity between the texts and guess the answer to the question by performing postselections. More details on question asking can be found [here](https://arxiv.org/pdf/2409.08777).

Now that we already have the circuits representing the texts, we need to make the circuits representing the questions. Remember, in our experiment, we need to have a pair of circuits, one for the affirmative case, and the other for the negative case. However, when adding the box corresponding to the question, we have to make sure that the wires of the question box correspond to the nouns from the text that are asked about.  

Below, the function `return_noun_list` returns all the nouns in a text and the function `return_q_nouns` return all the nouns in a question. One can notice that in the latter, we take the second and fifth noun as the subject and object of the question respectively. This works because of the simple case of the babI6 experiments, all the questions are of the format "Is the subject in the location?".

In [ ]:
from lambeq.backend.grammar import Ty

def return_noun_list(text):
    noun_list = []
    for b in text.boxes:
        if b.dom == Ty() and b.cod == N:
            noun_list.append(b.name)
            
    return noun_list

In [ ]:
def return_q_nouns(question):
    question_words = question.split(' ')
    q_nouns = [question_words[1], question_words[4].strip('?')]
    return q_nouns

We proceed to add the lists obtained from the functions above to the dictionary to be used later once we want to add the question asking boxes. 

In [ ]:
for i in datadict.keys():
    datadict[i].update({'noun_list_text': return_noun_list(datadict[i]['dsc_diag'])})
    datadict[i].update({'noun_list_question': return_q_nouns(datadict[i]['question'])})

We needed to extract the list of nouns in the texts as well as the list of nouns in their corresponding questions to remove the entries where the question nouns include nouns that are not in the text (questions that ask about subjects or locations not present in the text. We do this for simplification purposes). The following cell checks every entry from the `datadict` dictionary and adds the ids of "bad" entries (entries where the question contains nouns not present in the text) to the `discarded_ids` list that will be used later on.

In [ ]:
#reduce texts where question nouns are not in the text
discarded_datalist = []
discarded_ids = []
for i in datadict.keys():
    text_nouns = datadict[i]['noun_list_text']
    q_nouns = datadict[i]['noun_list_question']
    for noun in q_nouns:
        if noun not in text_nouns:
            discarded_datalist.append(datadict[i])
            discarded_ids.append(i)
            break

Now that we have a list of ids of entries that should be discarded because they contain nouns in the questions that are not present in the reduced contexts corresponding to them, we simply remove the said entries from the `datadict` dictionary.

In [ ]:
reduced_datadict = {}
for i in datadict.keys():
    if i not in discarded_ids:
        if datadict[i]['text_length'] < TEXT_LENGTH:
            reduced_datadict.update({i:datadict[i]})

Moreover, remember that for efficiency purposes, we also wanted to limit the number of wires in every circuit by chekcing that every circuit's codomain (which is the number of open wires of a circuit) is less than or equal to `MAX_WIDTH`. The following filters the entries in the `datadict` dictionary and only keep the entries in which the text circuits have less than or equal to  the maximum number of wires specified in `MAX_WIDTH`.

In [ ]:
# Reducing the size of the dictionary by removing the circuits that have more than a certain number of wires. 
def right_cod_size(circuit):
    if len(circuit.cod) > MAX_WIDTH:
        return False
    return True

def filter_entries_by_pos_neg_circuit_pair(entries):
    filtered_entries = {}
    
    for key, entry in entries.items():
        pos_neg_circuit_pair = entry.get('text_circuit_sim4_13')
        if pos_neg_circuit_pair and right_cod_size(pos_neg_circuit_pair):
            filtered_entries[key] = entry
    
    return filtered_entries

filtered_cod_datadict = filter_entries_by_pos_neg_circuit_pair(reduced_datadict)

Now that the text circuits are post-processed for optimization, it is time to make the question asking circuits to later sequentially compose the latter with the former. 

We first start with constructing two boxes `q1` and `q2` for both the affirmative and negative questions respectively. An affirmative question is just the same question that came with the text (in the pre-processing step). On the other hand, the negative question refers to the question on the negative case of the affirmative questions. For example, if the question related to a text is "Is Emily in the kitchen?", the equivalent negative question would be "Is Emily not in the kitchen?". For the purposes of this training, all the questions are either in the format of "Is subject in object?" or "Is subject not in object?". Therefore, we will need two boxes for the questions, a box for the "is in" questions, and another for the "is not in" questions. The purpose of having two generic boxes is that the ML model will learn later the parameters for these boxes. For more details about question asking in DiscoCir, we recommend [this paper](https://arxiv.org/pdf/2409.08777).

Notice that we also created two question boxes that are equiped with swaps, the purpose of which will become clearer in later parts of the tutorial. 

We apply the same ansatz applied on the text circuits (Sim4Ansatz with 3 layers and one qubit for each wire). Lastly, we add the postselections by sequentially composing the resulting circuit from applying the ansatz to a parallel composition of two effects (bras). 

In [ ]:
from lambeq.backend.grammar import Box, Id
from lambeq.backend.quantum import  Bra, Discard, qubit, Id, Swap
from lambeq import AtomicType, Sim4Ansatz

N = AtomicType.NOUN

q1 = Box('is_in', N@N, N@N)
q2 = Box('is_not_in', N@N, N@N)

ansatz = Sim4Ansatz({N:1}, n_layers=3)
qcirc1 = ansatz(q1)
qcirc2 = ansatz(q2)

#add the postselections to the questions
qcirc1_final = qcirc1 >> Bra(0) @ Bra(0)
qcirc2_final = qcirc2 >> Bra(0) @ Bra(0)

is_in_q = qcirc1_final
is_not_in_q = qcirc2_final

is_in_q_swp = Swap(qubit, qubit) >> qcirc1 >> Bra(0) @ Bra(0)
is_not_in_q_swp = Swap(qubit, qubit) >> qcirc2 >> Bra(0) @ Bra(0)

## 5. Assembling The Text Circuits with the Question Circuits

Now that we have all the ingredients in place (the text and question circuits), it is time to assemble them using sequential composition. However, we need to be careful and only attach the wires in the question boxes to the corresponding wires in the text boxes so that the nouns match. Moreover, we have to discard the wires of the nouns that are not included in the question. In order to do this, we might need to perform some swaps so that the wires that get composed with the question circuit are the corresponding wires from the text circuit.

We start by creating a layer composed of either identities (to link with the wires corresponding to the question nouns), or discards (for the rest of the wires). Once we sequentially compose this layer with the text circuit, this leaves us with a circuit whose codomain has two wires corresponding to the question. In order for us to attach the question boxes, we have to make sure that the wires from the question circuits are linked to the right wires from the text circuit. To achieve this, we check the question ids of the wires in the text circuits (to see whether the nouns in the text circuits are in the right order). This helps us decide whether to use the quetion boxes that come with swaps, or the ones without swaps (if the question wires are in the wrong order, we would need a swap to bring them back to the right order for the questions. Remember, we already created question boxes that are also equiped with swaps for this purpose).

Notice that, throughout the next cell, we always have two circuits. The circuit names ending in "pos" signal the circuits corrsponding to the affirmative questions, while their counterparts ending in "neg" signal the ones corresponding to the negative questions.

In [ ]:
# Bringing everything together by plugging the question asking part to the text circuits
for i in reduced_datadict.keys():
    
    text_circuit = reduced_datadict[i]['text_circuit_sim4_13']
    text_nouns = datadict[i]['noun_list_text']
    q_nouns = datadict[i]['noun_list_question']
    qid1 = datadict[i]['noun_list_text'].index(q_nouns[0])
    qid2 = datadict[i]['noun_list_text'].index(q_nouns[1])

    swap_required = False
    
    if qid1 > qid2: 
        swap_required = True
    elif qid1 < qid2:
        swap_required = False
    else:
        print('noun ids are idential, error')

    quest_mid_layer = Id(qubit) if (qid1 == 0 or qid2 == 0) else Discard()
    
 
    for k in range(1, len(text_circuit.cod)):
        if k == qid1 or k == qid2:
            quest_mid_layer = quest_mid_layer @ Id(qubit)
        else:
            quest_mid_layer = quest_mid_layer @ Discard()

    final_circuit = text_circuit >> quest_mid_layer

    if swap_required:
        final_circuit_pos = final_circuit >> is_in_q_swp
        final_circuit_neg = final_circuit >> is_not_in_q_swp
    else:
        final_circuit_pos = final_circuit >> is_in_q
        final_circuit_neg = final_circuit >> is_not_in_q

    reduced_datadict[i].update({'pos_neg_circuit_pair': (final_circuit_pos, final_circuit_neg)})

## 6. Preparing The Datasets for Training
Now that our circuit pairs are ready, we move on to the final step of training a model.

The first step is to prepare the data for training. We start with updating the "yes".

In [ ]:
babi6_datadict = {}
for i in reduced_datadict.keys():
    # Add the updated dictionary with the transformed 'answer'
    babi6_datadict.update({
        i: {
            'text': reduced_datadict[i]['text'],
            'question': reduced_datadict[i]['question'],
            'answer': 1 if reduced_datadict[i]['answer'] == 'yes' else 0,  # Transform 'yes' to 1 and 'no' to 0
            'quantum_circ_pair_pos_neg': reduced_datadict[i]['pos_neg_circuit_pair'],
            'text_length': reduced_datadict[i]['text_length']
        }
    })

The next step would be to make three sets: training, validation, and test sets. We try to balance the entries.

In [ ]:
import random
from collections import defaultdict

# Add the 'measure' field to each item
for key, value in babi6_datadict.items():
    temp = -1 if value['answer'] == 0 else 1
    value['measure'] = temp * value['text_length']

# Group items by absolute value of measure
abs_value_groups = defaultdict(list)
for key, value in babi6_datadict.items():
    abs_value = abs(value['measure'])
    abs_value_groups[abs_value].append((key, value))

# Balance signs within each group and ensure diverse sizes
new_balanced_dict = {}
for abs_value, items in abs_value_groups.items():
    # Separate positive and negative items
    positive_items = [(k, v) for k, v in items if v['measure'] > 0]
    negative_items = [(k, v) for k, v in items if v['measure'] < 0]
    
    # Determine the maximum balanced size for this group
    min_size = min(len(positive_items), len(negative_items))
    print("the minimum size is: " + str(min_size))
    
    # Randomly sample from each group to balance
    balanced_positive = random.sample(positive_items, min_size)
    balanced_negative = random.sample(negative_items, min_size)
    
    # Add to the balanced dictionary
    for k, v in balanced_positive + balanced_negative:
        new_balanced_dict[k] = v

the minimum size is: 88


In [ ]:
from sklearn.model_selection import train_test_split

# Convert dictionary into list of keys and values
keys = list(new_balanced_dict.keys())
values = list(new_balanced_dict.values())

# Split into training and temporary (validation + testing)
train_keys, temp_keys, train_values, temp_values = train_test_split(
    keys, values, test_size=0.4, random_state=42  # 60% training, 40% temp
)

# =Split the temporary set into validation and testing
val_keys, test_keys, val_values, test_values = train_test_split(
    temp_keys, temp_values, test_size=0.5, random_state=42  # 20% validation, 20% testing
)

# Reconstruct dictionaries for training, validation, and testing
training_dict_babi6 = {k: v for k, v in zip(train_keys, train_values)}
validation_dict_babi6 = {k: v for k, v in zip(val_keys, val_values)}
test_dict_babi6 = {k: v for k, v in zip(test_keys, test_values)}

In [ ]:
yes_count = 0
no_count = 0
for i in training_dict_babi6:
    if training_dict_babi6[i]['answer'] == 0:
        no_count += 1
    else:
        yes_count += 1

print(yes_count)
print(no_count)

52
53


Now, the final step is to store all of this data in separate files for training, validation, and testing, to be used in part II of this tutorial.

In [ ]:
with open(TRAINING_DATASET_FILEPATH, 'wb') as file:
    pickle.dump(training_dict_babi6, file)
with open(VALIDATION_DATASET_FILEPATH, 'wb') as file:
    pickle.dump(validation_dict_babi6, file)
with open(TEST_DATASET_FILEPATH, 'wb') as file:
    pickle.dump(test_dict_babi6, file) 